# Chapter 8: PID Controllers and Modified PID Controllers

## Example 8.1: Tuning_a_PID_controller_using_Nichols_Second_Rule.sce

In [ ]:
// Example 8-1
// Tuning a PID controller using Nichols Second Rule
clear; clc;
xdel(winsid());  //close all windows
mode(0);
// please edit the path
// cd <your code directory>
// exec('plotresp.sci');
// exec('rootl.sci');
s = %s;
G = 1 / ( s * (s + 1) * (s + 5) )
// finding Kcr and wcr (omega cr)
w = poly(0,'w');
D = horner(denom(G),%i * w);
x = roots(imag(D));
wcr = abs(x(2))  // the non zero root
Kcr = -1*clean(horner(D,wcr))
Pcr = 2*%pi / wcr 
Kp = 0.6 * Kcr
Ti = 0.5*Pcr
Td = 0.125*Pcr
Gc = Kp * ( s + 1/Ti + s^2*Td ) / s
GGc = syslin('c',G*Gc);
H = syslin('c',GGc /. 1);
disp(H,'closed loop system =');
rootl(GGc,0,'Root locus of open loop system');
sgrid([0.3],[]);
a = gca(); a.data_bounds = [-7 -4; 2 4];
xstring(-1,1,'zeta = 0.3');
scf();
t = 0:0.1:14;
u = ones(1,length(t));
plotresp(u,t,H,'');
// unacceptably large maximum overshoot
// new system
Kp2 = 39.42
Ti2 = 3.077
Td2 = 0.7692
Gc2 = Kp2 * ( s + 1/Ti2 + s^2*Td2 ) / s
GGc2 = syslin('c',G*Gc2);
H2 = syslin('c',GGc2 /. 1);
disp(H2,'closed loop system2 =');
plotresp(u,t,H2,'Step Response to a PID controlled system');
xstring(1.5,1.65,'System 1');
xstring(0.5,1.3,'System 2');

## Example 8.2: Computation_of_Optimal_solution_1.sce

In [ ]:
// Example 8-2
// Computation of Optimal solution 1
clear; clc;
xdel(winsid());  //close all windows
// please edit the path
// cd '';
// exec('plotresp.sci');
s = %s;
G = 1.2 / ( 0.36*s^3+ 1.86*s^2 + 2.5*s + 1);
K = 2.0 : 0.2 : 3.0;
a = 0.5 : 0.2 : 1.5;
t = 0:0.1:5; u = ones(1,length(t));
// lesser points for a rough check
t1 = 0:0.01:5; u1 = ones(1,length(t1));
// more points for a rigorous check
k = 0;
for i = 1:6
  for j = 1:6
    Gc = K(i) * (s + a(j))^2 / s;
    H = G * Gc;
    H = syslin('c', H /. 1);
    y = csim(u,t,H); 
    m = max(y);
    if m < 1.1 then 
      y = csim(u1,t1,H);
      m = max(y);
      if m  < 1.1 then
        k = k + 1; 
        solution(k,:) = [K(i) a(j) m];
      end
    end
  end
end
disp(solution,'solution [K a m] = ');
// to sort the matrix
[x O] = gsort(solution(:,3),'r','i');
// re order the matrix
for i = 1:k
  sortsolution(i,:) = solution( O(i) , :);
end
disp(sortsolution,'sortsolution [K a m] = '); 
// Response with largest overshoot above 10%
x = sortsolution(k,:);
K = x(1); a = x(2); 
Gc = K * (s + a)^2 / s;
H = G * Gc;
H = syslin('c', H /. 1);
plotresp(u1,t1,H,'Step Response with 10% overshoot');
disp(Gc,'Gc = ');
disp(H,'H = ');
  

## Example 8.3: Computation_of_Optimal_solution_2.sce

In [ ]:
// Example 8-3
// Computation of Optimal solution 2
clear; clc;
xdel(winsid());  //close all windows
// please edit the path
// cd '';
// exec('plotresp.sci');
s = %s;
G = 4 / ( s^3+ 6*s^2 + 8*s + 4);
t = 0:0.1:8; u = ones(1,length(t));
// lesser points for a rough check
t1 = 0:0.01:8; u1 = ones(1,length(t1));
// more points for a rigorous check
k = 0;
mprintf('Processing...
');
for K = 3:0.2:6
  for a = 0.1:0.1:3
    Gc = K * (s + a)^2 / s;
    H = G * Gc;
    H = syslin('c', H /. 1);
    y = csim(u,t,H);
    m = max(y);
    if m < 1.15 & m > 1.08 then
       // give  a margin of 0.02 for the rough check - 1.08 
       y = csim(u1,t1,H);
       m = max(y);
       if m < 1.15 & m > 1.10 then
         // check for settling time
         l =length(t1);
         while y(l) > 0.98 & y(l) < 1.02 ; l = l-1; end
            ts = (l-1) * 0.01;
         if ts < 3.00 then   
            k = k + 1;
            solution(k,:) = [K a m ts]; 
          end
        end
      end
      
    end
  if modulo(K*10,2) == 0 then mprintf(' completed %d%%
', (K - 3)/3*100)
 end
end
  
disp(solution,'solution [K a m ts] = ');
[x O] = gsort(solution(:,3),'r','i');
for i = 1:k
  sortsolution(i,:) = solution( O(i) , :);
end
disp(sortsolution,'sortsolution [K a m ts] = '); 
// Response with smallest overshoot
x = sortsolution(1,:);
K = x(1); a = x(2); 
Gc = K * (s + a)^2 / s;
H = G * Gc;
H = syslin('c', H /. 1);
plotresp(u,t,H,'Step Response with smallest overshoot');
disp(Gc,'Gc = ');
disp(H,'H = '); 
 

## Example 8.4: Design_of_system_with_two_degrees_of_freedom.sce

In [ ]:
// Example 8-4
// Design of system with two degrees of freedom
clear; clc;
xdel(winsid());  //close all windows
mode(0);
// please edit the path
// cd '';
// exec('plotresp.sci');
s = %s;
// Design Step 1: choosing a, b and c.
t = 0:0.1:4;
u = ones(1,length(t));
t1 = 0:0.01:4;
N = length(t1);
u1 = ones(1,N);
// N = N - 3
k = 0;
mprintf('Processing...
');
for i = 1:21
  a = 6.2 - 0.2*i;
  for j = 1:21
    b = 6.2 - 0.2*j;
    for h = 1:21
      c = 12.2 - 0.2*h; 
      num = (2*a + c)*s^2 + (a*a + b*b + 2*a*c)*s + (a*a + b*b)*c;
      den = s^3 + num;
      G = syslin('c',num,den);      
      y = csim(u,t,G);
      m = max(y);
      if m < 1.19 & m > 1.00 then
        y = csim(u1,t1,G);
        m = max(y);
        if m < 1.19 & m > 1.02 then
          l = N;
          while y(l) > 0.98 & y(l) < 1.02 ; l = l-1; end
            ts = (l-1) * 0.01; 
          if ts < 1.0 then
             k = k + 1;
             solution(k,:) = [a b c m ts];
           end
         end
       end
             
     end
  end
 mprintf(' completed %d%%
', (6 - a)/4*100);
end
  
disp(solution,'solution = ');
K = solution(1,:);
a = K(1); b = K(2); c = K(3);
num = (2*a + c)*s^2 + (a*a + b*b + 2*a*c)*s + (a*a + b*b)*c;
den = s^3 + num;
YbyR = syslin('c',num,den);disp(YbyR,'Y(s)/R(s) =');
subplot(2,1,1);
plotresp(u1,t1,YbyR,'Step response for a = 4.2 ,b = 2 ,c =12');
cf  = coeff(den);
K = (cf(3) - 1) / 10 
alpha_plus_beta = cf(2) / K /10
alphabeta = cf(1) / K / 10
Gc = K * (s^2 + alpha_plus_beta*s + alphabeta) / s
YbyD = syslin('c',10*s,den);
disp(YbyD,'Y(s)/D(s) = ');
subplot(2,1,2);
plotresp(u1,t1,YbyD,'Response to step disturbance input for a = 4.2 ,b = 2 ,c =12');
a = gca(); a.data_bounds = [0 -0.01; 4 0.07];
// Design Step 2
scf();
Gc1 = (YbyR.num / 10) / s
Gc2 = Gc - Gc1
// response to reference inputs
y1 = csim(t,t,YbyR); u = 1/2 * t.^2;
y2 = csim(u,t,YbyR);
subplot(2,1,1);
plotresp(t,t,YbyR,'Response to unit ramp input');
subplot(2,1,2);
plotresp(u,t,YbyR,'Response to unit acceleration input');

## Example 8.5: Design_of_system_with_two_degrees_of_freedom_2.sce

In [ ]:
// Example 8-5
// Design of system with two degrees of freedom 2
clear; clc;
xdel(winsid());  //close all windows
mode(0);
// please edit the path
// cd '';
// exec('plotresp.sci');
s = %s;
Gp = 5 / (s+1) / (s+5)
t = 0:0.01:3;
u = ones(1,length(t)); 
// Step 1: Design of Gc1
a = sqrt(13)
K = 4 / (5*a - 15)
Gc1 = K * (s + a)^2 / s
// determining Kp. Ti and Td
cf = coeff(numer(Gc1));
Kp = cf(2)
Ti = Kp / cf(1)
Td = cf(3) / Kp
subplot(2,1,1);
YbyD  = syslin('c',Gp / (1 + Gp * Gc1))
plotresp(u,t,YbyD,'Response to step disturbance input');
a = gca();
a.data_bounds = [0 0; 3 0.1];
//Step 2: Design of Gc2
cf = coeff(YbyD.den);
Kp2 = (cf(2) - 47.63 ) / 5
Td2 = (cf(3) - 6.6051) / 5 / Kp2
Gc2 = Kp2 * ( 1 + Td2*s)
YbyR = syslin('c',1 - s^3 / YbyD.den )
subplot(2,1,2);
t = 0:0.05:2;
u = ones(1,length(t)); 
plotresp(u,t,YbyR,'Response to step reference input');
scf();
subplot(2,1,1);
plotresp(t,t,YbyR,'Response to ramp reference input');
subplot(2,1,2);
u = 1/2 * t.^2;
plotresp(u,t,YbyR,'Response to acceleration reference input');

## Example 8.a_12: Computing_optimal_solution.sce

In [ ]:
// Example A-8-12
// Computing optimal solution
clear; clc;
xdel(winsid());  //close all windows
s = %s;
t = 0:0.1:5; u = ones(1,length(t));
t1 = 0:0.01:5;N =length(t1);  u1 = ones(1,N);
k = 0;
mprintf('Processing ...
');
for K = 50:-1:2
  for a = 2:-0.05:0.05
    num = K * ((s + a)^2) ;
    den = s * s * (s^2 + 6*s + 5);
    G = syslin('c',num,num + den);
    y = csim(u,t,G);
    m = max(y);
    if m < 1.1 & m > 1.00 then;
      y = csim(u1,t1,G);  
      if m < 1.1 & m > 1.02 then; 
        l = N;
        while y(l) > 0.98 & y(l) < 1.02 ; l = l-1; end 
        ts = (l-1)*0.01;
        if ts < 3.0;
          k= k + 1;
          solution(k,:) = [K a m ts];
        end
      end
    end
    
  end
 mprintf('completed %d%%
',(50 - K)/48*100);
end      
disp(solution,'solution = ');
// sort the solution set
[x O] = gsort(solution(:,3),'r','i');
for i = 1:k
  sortsolution(i,:) = solution(O(i),:);
end
disp(sortsolution,'sortsolution = ');
x = sortsolution(7,:); K = x(1); a = x(2)
    num = K * ((s + a)^2) ;
    den = s * s * (s^2 + 6*s + 5);
    G = syslin('c',num,num + den);
    y1 = csim('step',t1,G);
x = sortsolution(2,:); K = x(1); a = x(2)
    num = K * ((s + a)^2) ;
    den = s * s * (s^2 + 6*s + 5);
    G = syslin('c',num,num + den);
    y2 = csim('step',t1,G);
plot(t1,y1,t1,y2);
xgrid();
xtitle('Unit Step response curves','t (sec)','output');
legend('K = 29 , a = 0.25', 'K = 27 , a = 0.2');

## Example 8.a_13: Design_of_system_with_two_degrees_of_freedom.sce

In [ ]:
// Example A-8-13
// Design of system with two degrees of freedom
clear; clc;
xdel(winsid());  //close all windows
mode(0);
// please edit the path
// cd '<path to dependencies';
// exec('plotresp.sci');
s = %s;
Gp = 100 /(s*(s + 1))
dp = -5 + %i*5;
// Step 1: Design of Gc1 using root locus approach
angdef = 180 - phasemag(horner(Gp/s,dp))
angdef2 = angdef /2;
disp(angdef2,'each pole must contribute an angle of');
a = 5 + 5*cotd(angdef2)
Gcx = (s + a)^2 / s;
K = 1/ abs(horner(Gcx*Gp, dp) )
Gc1 = K * (s + a)^2 / s
// determining Kp. Ti and Td
cf = coeff(numer(Gc1));
Kp = cf(2)
Ti = Kp / cf(1)
Td = cf(3) / Kp
t = 0:0.01:4;
u = ones(1,length(t)); 
subplot(2,1,1);
YbyD  = syslin('c',Gp / (1 + Gp * Gc1))
plotresp(u,t,YbyD,'Response to step disturbance input');
ax = gca();
ax.data_bounds = [0 0; 3 2];
//Step 2: Design of Gc
Gc = (YbyD.den - s^3) / 100 /  s
YbyR = syslin('c',1 - s^3 / YbyD.den )
subplot(2,1,2);
t = 0:0.01:3;
u = ones(1,length(t)); 
plotresp(u,t,YbyR,'Response to step reference input');
scf();
subplot(2,1,1);
plotresp(t,t,YbyR,'Response to ramp reference input');
subplot(2,1,2);
t = 0:0.01:2;
u = 1/2 * t.^2;
plotresp(u,t,YbyR,'Response to acceleration reference input');

## Example 8.a_5: PID_desig.sce

In [ ]:
// Example A-8-5
// PID design
clear; clc;
xdel(winsid());  //close all windows
mode(0);
// please edit the path
// cd '';
// exec('plotresp.sci');
// exec('stepch.sci');
s = %s;
zeta = 0.5       // dominant pole charecteristics
wn = 4
sigma = zeta*wn;
ts = 4 /(zeta*wn); 
disp(ts,'settling time approximate (ts) =');
D = (s + 10) * (s^2 + 2*zeta*wn*s + wn^2);
cf = coeff(D);
K = cf(1)
a_plus_b  = (cf(2) - 9) / K
ab = (cf(3) - 3.6) / K
Gc = K * (ab * s^2 + a_plus_b *s+ 1) / s
CbyD = syslin('c',s,D)
CbyR = syslin('c',numer(Gc),D)
t = 0:0.05:5;
u = ones(1,length(t));
plotresp(u,t,CbyD,'Response to step disturbance input');
a = gca(); a.data_bounds = [0 ,-4D-3; 5 ,14D-3];
scf();
[Mp ,tp ,tr ,ts] = stepch(CbyR,0,5,0.05,0.02);
disp(Mp,'Max overshoot ='); 
disp(ts,'settling time actual (ts) =');

## Example 8.a_6: PID_desig.sce

In [ ]:
// Example A-8-6
// PID Design
clear; clc;
xdel(winsid());  //close all windows
mode(0);
// please edit the path
// cd '<your code directory>';
// exec('plotresp.sci');
// exec('rootl.sci')
s = %s;
G = syslin('c',1,s^2 + 1);
dp = -1 + sqrt(3)*%i;
angdef = 180 - phasemag(horner(G*(s+1)/s,dp)) 
// Determining b
b = 1 + sqrt(3)*cotd(angdef)
Gc1 = (s + 1) * (s + b) / s;
K = 1/ abs(horner(G*Gc1,dp))
Gc = K * Gc1
evans(G*Gc1,50);
xgrid();
a = gca(); 
a.data_bounds = [-5 -3; 1 3];
a.children(1).visible = 'off';
xtitle('Root locus plot of open loop system'); 
C = syslin('c',G*Gc /. 1);
disp(C,'closed loop system =');
scf();
t = 0:0.05:12;
u = ones(1,length(t));
plotresp(u,t,C,'Unit step response of compensated system');

## Example 8.a_7_1: PID_Design_with_Frequency_Response.sce

In [ ]:
// Example A-8-7-1
// PID Design with Frequency Response
clear; clc;
xdel(winsid());  //close all windows
mode(0);
// please edit the path
// cd '<your code directory>';
// exec('plotresp.sci');
s = %s;
Gp = syslin('c',s + 0.1,s^2 + 1);
Kv = 4;
K =  Kv / abs(horner(Gp,0))
// Step 1 : Gain adjust
G1 = Gp * K / s
G1w = syslin('c', horner(G1, %s/2/%pi) );// correction for frequences in rad/s
subplot(2,1,1);  bode(G1w);
xtitle('Bode plot of G(s) = 40*(s + 0.1)/ [s*(s^2 + 1)]','rad/s');
a = gcf();set(a.children(1).x_label,'text','rad/s');
disp(p_margin(G1w),'Phase margin of G =');
// Step 2:
a = 0.1526;
GGc = G1 * (a*s + 1)
GGcw = syslin('c', horner(GGc, %s/2/%pi) );
subplot(2,1,2);  bode(GGcw,0.1,10);
xtitle('Bode plot of G*Gc = [4 *(0.1526*s + 1)*(s + 0.1)]/[s*(s^2 + 1)]','rad/s');
a = gcf();set(a.children(1).x_label,'text','rad/s');
disp(p_margin(GGcw),'Phase margin of G*Gc =');
disp(g_margin(GGcw),'Gain margin of G*Gc =');
scf();
C  = syslin('c',GGc /. 1)
disp(roots(C.den),'closed loop poles =');
t = 0:0.05:10;
u = ones(1,length(t));
subplot(2,1,1); plotresp(u,t,C,'Step response of PID controlled system');
subplot(2,1,2); plotresp(t,t,C,'Ramp response of PID controlled system');

## Example 8.i_1: PID_Design_with_Frequency_Response.sce

In [ ]:
// Illustration 8.1
// PID Design with Frequency Response
clear; clc;
xdel(winsid());  //close all windows
mode(0);
// please edit the path
// cd '<your code directory>';
// exec('plotresp.sci');
s = %s;
G = syslin('c',1,s^2 + 1);
Kv = 4;
K =  Kv / abs(horner(G,0))
// Step 1 : Gain adjust
G2 = G * K / s
G2w = syslin('c', horner(G2, %s/2/%pi) );// correction for frequences in rad/s
omega = calfrq(G2w,0.1,10);   // discretises such that the peak is                                           // well represented
[db phi] = dbphi(repfreq(G2w,omega));
phi( 53:99 ) = -270;
subplot(2,1,1);  bode(omega,db,phi);
xtitle('Bode plot of G(s) = 4 / [s * (s^2 + 1)]','rad/s');
a = gcf();set(a.children(1).x_label,'text','rad/s');
disp(p_margin(G2w),'Phase margin of G2 =');
// Step 2:
a = 5 // a is chosen to be 5;
G3 = G2 * (a*s + 1)
G3w = syslin('c', horner(G3, %s/2/%pi) );
subplot(2,1,2);  bode(G3w,0.1,10);
xtitle('Bode plot of G(s) = [4 * (5*s + 1)] / [s * (s^2 + 1)]','rad/s');
a = gcf();set(a.children(1).x_label,'text','rad/s');
disp(p_margin(G3w),'Phase margin of G3 =');
// Step 3
scf();
b = 0.25
G4 = G3 * (b*s + 1)
G4w = syslin('c', horner(G4, %s/2/%pi) );
subplot(2,1,1);  bode(G4w,0.1,10);
xtitle('Bode plot of G(s) = [4 * (5*s + 1)] / [s * (s^2 + 1)*(0.25*s +1)]','rad/s');
a = gcf();set(a.children(1).x_label,'text','rad/s');
disp(p_margin(G4w),'Phase margin of G4 =');
C  = syslin('c',G4 /. 1)
disp(roots(C.den),'closed loop poles =');
t = 0:0.1:14;
u = ones(1,length(t));
subplot(2,1,2); plotresp(u,t,C,'Step response of PID controlled system');